In [3]:
import pandas as pd
school_df = pd.read_csv('../data/social_explorer_raw/School_Digger_2018.csv')
school_df = school_df.drop(index=0)
school_df = school_df.reset_index(drop=True)
print(school_df.shape)
school_df = school_df[['Qualifying Name', 'SchoolDigger Rank Elementary', 'SchoolDigger Rank High', 'SchoolDigger Rank Middle']]
school_df.head()
# school_df.columns

(19349, 30)


,Qualifying Name,SchoolDigger Rank Elementary,SchoolDigger Rank High,SchoolDigger Rank Middle
0,"Alabama Youth Services, Alabama",NaN,NaN,NaN
1,"Albertville City, Alabama",363,203,134
2,"Marshall County, Alabama",422,172.75,246
3,"Hoover City, Alabama",116.888888888889,12,22.3333333333333
4,"Madison City, Alabama",31.1428571428571,8,15.5


In [4]:
crime_df = pd.read_csv('../data/social_explorer_raw/crime.csv')
crime_df = crime_df.drop(index=0)
crime_df = crime_df.reset_index(drop=True)
crime_df['Total Violent and Property Crimes Rate'] = crime_df['Total Violent and Property Crimes Rate'].astype('float')
print(crime_df.shape)
# filtered_df = crime_df[crime_df['Name of Area'] == 'Crittenden County']
crime_df = crime_df[['Qualifying Name', 'Total Violent and Property Crimes Rate']]

crime_df = crime_df.sort_values(by='Total Violent and Property Crimes Rate', ascending=False)
print(crime_df['Total Violent and Property Crimes Rate'].isna().sum())
filtered_df = crime_df[pd.isna(crime_df['Total Violent and Property Crimes Rate'])]
# there are 42 counties that have no crime rate, will spot check these and see if we can assume that means they have very low crime/pop
# crime_df.columns
filtered_df.head(100)
# crime_df.head(3300)

KeyError: 'Total Violent and Property Crimes Rate'

In [5]:
health_df = pd.read_csv('../data/social_explorer_raw/Health_data_2023_Release.csv')
health_df = health_df.drop(index=0)
health_df = health_df.reset_index(drop=True)
# print(health_df.shape)
# health_df.head()
health_df.columns

Index(['Qualifying Name', 'Name of Area', 'FIPS', 'Nation', 'State', 'County',
       'Physically Unhealthy Days per Month (Persons 18 Years and Over)',
       'Mentally Unhealthy Days per Month (Persons 18 Years and Over)',
       'Percent of Adults That Report Fair or Poor Health (Persons 18 Years and Over)',
       'Percent of Low Birthweight Births (<2.5kg)',
       'Primary Care Physicians (PCP)', 'Mental Health Providers (MHP)',
       'Dentists', 'Primary Care Physicians (PCP) Rate per 100,000 Population',
       'Mental Health Providers (MHP) Rate per 100,000 Population',
       'Dentists Rate per 100,000 Population',
       'Percent of Persons Without Insurance (Population Under 19 Years, 2013 est.)',
       'Percent of Persons Without Insurance (Population Under 65 Years, 2013 est.)',
       'Persons Without Insurance (Population Under 19 Years, 2013 est.)',
       'Persons Without Insurance (Population Under 65 Years, 2013 est.)',
       'Child Mortality (Death Counts)',
   

In [6]:
demographic_df = pd.read_csv('../data/social_explorer_raw/Census2020_PL94_Redistricting_Data.csv')
demographic_df = demographic_df.drop(index=0)
demographic_df = demographic_df.reset_index(drop=True)
# print(demographic_df.shape)
# demographic_df.head()
demographic_df.columns

Index(['Qualifying Name', 'Name of Area', 'FIPS', 'Nation', 'State', 'County',
       'County Subdivision', 'Census Tract', 'Block Group', 'Place',
       'Congressional District (116th)',
       'State Legislative District (Upper Chamber)',
       'State Legislative District (Lower Chamber)',
       'Metropolitan Statistical Area/Micropolitan Statistical Area',
       'Combined Statistical Area', '2020 Total Population',
       '2010 Total Population', 'Population Change',
       'Percent Change Population', '2020 Total Population.1',
       '2020 Population Density', '2010 Total Population.1',
       '2010 Population Density', 'Total Population Change',
       'Population Density Change', 'Total Population Change Percent',
       'Population Density Change Percent', '2020 Housing Units',
       '2020 Occupied Units', '2020 Vacant Units', '2010 Housing Units',
       '2010 Occupied Units', '2010 Vacant Units', 'Housing Units Change',
       'Occupied Units Change', 'Vacant Units Chang

In [7]:
housing_df = pd.read_csv('../data/social_explorer_raw/Income_limits_and_fair_market_rent_2020.csv')
housing_df = housing_df.drop(index=0)
housing_df = housing_df.reset_index(drop=True)
# print(housing_df.shape)
# housing_df.head()
housing_df.columns

Index(['Qualifying Name', 'Name of Area', 'FIPS', 'Nation',
       'Metropolitan Statistical Area', 'County/County Subdivision',
       'Total Population 2017', 'Median Family Income (5-year ACS)',
       'Low Income Limits (Up to 80% of Local Area Median Income) for One Person Families',
       'Low Income Limits (Up to 80% of Local Area Median Income) for Two Person Families',
       'Low Income Limits (Up to 80% of Local Area Median Income) for Three Person Families',
       'Low Income Limits (Up to 80% of Local Area Median Income) for Four Person Families',
       'Low Income Limits (Up to 80% of Local Area Median Income) for Five Person Families',
       'Low Income Limits (Up to 80% of Local Area Median Income) for Six Person Families',
       'Low Income Limits (Up to 80% of Local Area Median Income) for Seven Person Families',
       'Low Income Limits (Up to 80% of Local Area Median Income) for Eight Person Families',
       'Very Low Income Limits (Up to 50% of Local Area Me

In [8]:
#economic indicator
unemployment_df = pd.read_csv('../data/social_explorer_raw/Unemployment_Statistics_2021.csv')
unemployment_df = unemployment_df.drop(index=0)
unemployment_df = unemployment_df.reset_index(drop=True)
# print(unemployment_df.shape)
# unemployment_df.head()
unemployment_df.columns

Index(['Qualifying Name', 'Name of Area', 'FIPS', 'Nation', 'State', 'County',
       'Metropolitan Statistical Area', 'Labor Force as of July 2021',
       'Employed as of July 2021', 'Unemployed as of July 2021',
       'Unemployment rate % as of July 2021', 'Labor Force as of July 2020',
       'Employed as of July 2020', 'Unemployed as of July 2020',
       'Unemployment rate % as of July 2020',
       'Over the Year Labor Force change July 2020 - July 2021',
       'Over the Year Employed change July 2020 - July 2021',
       'Over the Year Unemployed change July 2020 - July 2021',
       'Over the Year Unemployed change July 2020 - July 2021.1',
       'Labor Force as of Jul', 'Employed', 'Unemployed as of Jul',
       'Unemployment rate %',
       'Over the Year Labor Force change June 2021 - July 2021',
       'Over the Year Employed change June 2021 - July 2021',
       'Over the Year Unemployed change June 2021 - July 2021',
       'Over the Year Unemployed change June 2021 -

In [9]:
#religion indicator
religion_df = pd.read_csv('../data/social_explorer_raw/Religion.csv', low_memory=False)
religion_df = religion_df.drop(index=0)
religion_df = religion_df.reset_index(drop=True)
# print(religion_df.shape)
# religion_df.head()
religion_df.columns

Index(['FIPS', 'Name of Area', 'Qualifying Name', 'Nation', 'State', 'County',
       'Metropolitan Statistical Area/Micropolitan Statistical Area',
       'Population (2020)', 'All Religious Congregations',
       'All Religious Adherents',
       ...
       'Reconstructionist Judaism.1', 'Reform Judaism.1', 'Shinto.1', 'Sikh.1',
       'Swedenborgian Church.1', 'Tao.1',
       'Union of Messianic Jewish Congregations.2',
       'Unitarian Universalist Association of Congregations.2',
       'Unity Churches', 'Zoroastrian.1'],
      dtype='object', length=695)

In [11]:
#environmental indicator
environment_df = pd.read_csv('../data/social_explorer_raw/Environmental_summaries_2011.csv')
environment_df = environment_df.drop(index=0)
environment_df = environment_df.reset_index(drop=True)
# print(environment_df.shape)
# environment_df.head()
environment_df.columns

Index(['FIPS', 'Name of Area', 'Qualifying Name', 'Nation', 'State', 'County',
       'Census Tract', 'Total Area of Land Coverage',
       'Area Of Land Cover Class 11, Open Water',
       'Area Of Land Cover Class 12, Perennial Ice/Snow',
       'Area Of Land Cover Class 21, Developed, Open Space',
       'Area Of Land Cover Class 22, Developed, Low Intensity',
       'Area Of Land Cover Class 23, Developed, Medium Intensity',
       'Area Of Land Cover Class 24, Developed, High Intensity',
       'Area Of Land Cover Class 31, Barren Land (Rock/Sand/Clay)',
       'Area Of Land Cover Class 41, Deciduous Forest',
       'Area Of Land Cover Class 42, Evergreen Forest',
       'Area Of Land Cover Class 43, Mixed Forest',
       'Area Of Land Cover Class 52, Shrub/Scrub',
       'Area Of Land Cover Class 71, Grassland/Herbaceous',
       'Area Of Land Cover Class 81, Pasture/Hay',
       'Area Of Land Cover Class 82, Cultivated Crops',
       'Area Of Land Cover Class 90, Woody Wetlands'

In [12]:

merged_df = pd.merge(crime_df, school_df, on='Qualifying Name', how='left')

# print(merged_df.shape)
merged_df.head()
# merged_df.columns

,Qualifying Name,Name of Area,FIPS,Nation,State,County,Total Population (2019 est.),Total Violent and Property Crimes,Total Violent and Property Crimes: Violent Crimes,Total Violent and Property Crimes: Property Crimes,...,Arsons,Total Number of Hate Crime Reported:,"Total Number of Hate Crimes with an Anti-Race, Anti-Ethnicity, Anti-Ancestry Bias Reported:",Total Number of Hate Crimes with an Anti-Religion Bias Reported:,Total Number of Hate Crimes with an Anti-Sexual Orientation Bias Reported:,Total Number of Hate Crimes with an Anti-Disability Bias Reported:,Total Number of Hate Crimes with an Anti-Gender and Anti-Gender Identity Bias Reported:,SchoolDigger Rank Elementary,SchoolDigger Rank High,SchoolDigger Rank Middle
0,"Autauga County, Alabama",Autauga County,01001,00,01,001,55869,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,275.8,157,198.75
1,"Baldwin County, Alabama",Baldwin County,01003,00,01,003,223234,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,186.045454545455,85.625,89.1818181818182
2,"Barbour County, Alabama",Barbour County,01005,00,01,005,24686,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,657,333,422
3,"Bibb County, Alabama",Bibb County,01007,00,01,007,22394,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,432,161,279
4,"Blount County, Alabama",Blount County,01009,00,01,009,57826,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,286.285714285714,164.714285714286,206.714285714286
